In [1]:
import pandas as pd
from enum import Enum, IntEnum
from pydantic import BaseModel
from openai import OpenAI,AsyncOpenAI
import json
import asyncio

from constants import LLM_com_path,LLM_org_path,VLM_com_path,VLM_org_path,ConfidenceScore,Confidence

In [2]:

async_client = AsyncOpenAI(api_key='api_key',timeout=60,max_retries=5)
# df = pd.read_csv(LLM_com_path)
df = pd.read_csv('completed.csv')

In [3]:
## all rows of df
column_names = list(df.columns)
raw_columns = [col for col in column_names if col.endswith('_rawoutput')]
confidence_sentence_columns = [col for col in column_names if col.endswith('_confidence-sen')]
confidence_number_columns = [col for col in column_names if col.endswith('_confidence-num')]
# printing all to check
print(raw_columns)
print(confidence_sentence_columns)
print(confidence_number_columns)


['ClaudeHiakuWeb-raw_rawoutput', 'ClaudeSonnetWeb-raw_rawoutput', 'ClaudeOpuWeb-raw_rawoutput', 'GeminiWeb-raw_rawoutput', 'GeminiAdvancedWeb-raw_rawoutput', 'gpt-3.5-turbo-0125_rawoutput', 'gpt-4-0613_rawoutput', 'GPT3.5Web_rawoutput', 'GPT4Web_rawoutput', 'llama2-13B-Q5KM_rawoutput', 'medicine-chat-Q8_rawoutput', 'llama2-7B-Q8_rawoutput', 'mistral-instruct-v2-Q8_rawoutput', 'phi3-3b-Q16_rawoutput', 'llama3-8b-Q8_rawoutput', 'Llama27B-Poe_rawoutput', 'Mixtral-8x7B-Poe_rawoutput', 'MistralLarge-Poe_rawoutput', 'Llama270B-Poe_rawoutput', 'claude-3-opus-20240229_rawoutput', 'claude-3-haiku-20240307_rawoutput', 'claude-3-sonnet-20240229_rawoutput', 'Llama3-70b-Poe_rawoutput', 'Llama3-8b-Poe_rawoutput', 'gpt-4o-2024-05-13_rawoutput', 'openbioLLM-7B-Q8_rawoutput', 'claude-3-5-sonnet-20240620_rawoutput', 'gpt-4o-mini-2024-07-18_rawoutput', 'gemma2-9b-it_rawoutput', 'Gemma2-9b-Q8_rawoutput', 'Mistral-7B-T_rawoutput', 'Llama-2-13b_rawoutput', 'Phi3-medium14b-Q6_rawoutput', 'Llama3.1-405B_rawou

In [84]:
len(confidence_number_columns)

37

## testing area

In [6]:
test = df.loc[2,confidence_sentence_columns[13]]
print(test)

My confidence in this decision is a 9 out


## extraction Function


In [7]:
# def extract_confidence(text):
#     completion = client.beta.chat.completions.parse(
#         model="gpt-4o-mini-2024-07-18",
#         messages=[
#             {"role": "system", "content": "you are a confidence score extractor, for geiven text please extract the confidence mentioned in text, if there are both number confidence and qualitative confidence please extract the number confidence. please just if confidence is mentioned pick a value for confidence and if it is not mentioned pick not_mentioned."},
#             {"role": "user", "content": text},
#         ],
#         response_format=ConfidenceScore,
#     )
#     return json.loads(completion.choices[0].message.content)

async def async_extract_confidence(text):
    completion = await async_client.beta.chat.completions.parse(
        model="gpt-4o-mini-2024-07-18",
        messages=[
            {"role": "system", "content": "you are a confidence score extractor, for geiven text please extract the confidence mentioned in text, if there are both number confidence and qualitative confidence please extract the number confidence. please just if confidence is mentioned pick a value for confidence and if it is not mentioned pick not_mentioned."},
            {"role": "user", "content": text},
        ],
        response_format=ConfidenceScore,
    )
    return json.loads(completion.choices[0].message.content)


In [9]:
lenn = len(df)

In [12]:
async def process_cell_conf_sen(txt):
    if pd.isna(txt):
        completion =''
    else:
        completion = await async_extract_confidence(txt)
    return completion
    
    #confidence_sentence_columns

In [13]:
def create_async_tasks(df,i):
    tasks = []
    for u in range(lenn):
        value = df.loc[u,confidence_sentence_columns[i]]
        tasks.append(asyncio.create_task(process_cell_conf_sen(value)))
    return tasks 

def save_to_df(res,kll):
    for i,r in enumerate(res):
        if r:
            df.loc[i,confidence_number_columns[kll]] = r['confidence']
        else:
            df.loc[i,confidence_number_columns[kll]] = ''

In [358]:
iii = 36

In [359]:
for u in range(lenn):
    value = df.loc[u,confidence_sentence_columns[iii]]
    if pd.isna(value):
        print("kir")
    else:
        print(value)


**Expert Judgment and Confidence:**   **Confidence Level: 8/10**  **Rationale:**  
**Expert Judgment and Confidence:**   **  **Confidence Level: 9/10**  **Rationale:**  
**Expert Judgment and Confidence:**   **  **Confidence Level: 8/10**  **Rationale:**  
**Decision and Confidence:**  Based on the patient's history, the size of the previous adenoma, and current guidelines, I would recommend a follow-up colonoscopy in **1 year (Option B)**.   **Confidence Level:** 9/10  **Rationale:**   My confidence is high because the patient's case fits squarely within the established guidelines for surveillance after EMR of large adenomas.
**Confidence Level:** 9/10  I am highly confident in this decision based on the patient's presentation, response to budesonide, and established treatment guidelines for Crohn's disease.   **Confidence Level:** 9/10  I am highly confident in this decision based on the patient's presentation, response to budesonide, and established treatment guidelines for Crohn's 

In [360]:
tasks = create_async_tasks(df,iii)

In [361]:
print(len(tasks))

300


In [362]:
results = await asyncio.gather(*tasks)

In [363]:
print(results)

[{'confidence': '8'}, {'confidence': '9'}, {'confidence': '8'}, {'confidence': '9'}, {'confidence': '9'}, {'confidence': '7'}, {'confidence': '8'}, {'confidence': '6'}, {'confidence': '9'}, {'confidence': '8'}, {'confidence': '8'}, {'confidence': '8'}, {'confidence': '9'}, {'confidence': '9'}, {'confidence': '9'}, {'confidence': '9'}, {'confidence': '8'}, {'confidence': '9'}, {'confidence': '8'}, {'confidence': '9'}, {'confidence': '9'}, {'confidence': '9'}, {'confidence': '9'}, {'confidence': '8'}, {'confidence': '9'}, {'confidence': '8'}, {'confidence': '8'}, {'confidence': '9'}, {'confidence': '7'}, {'confidence': '9'}, {'confidence': '9'}, {'confidence': '10'}, {'confidence': '7'}, {'confidence': '9'}, {'confidence': '8'}, {'confidence': '9'}, {'confidence': '9'}, {'confidence': '9'}, {'confidence': '9'}, {'confidence': '9'}, {'confidence': '10'}, {'confidence': '8'}, {'confidence': '9'}, {'confidence': '8'}, {'confidence': '9'}, {'confidence': '9'}, {'confidence': '9'}, {'confiden

In [364]:
print(len(results))

300


In [ ]:
save_to_df(results,iii)

In [367]:
df.to_csv("completed.csv",index=False)

In [368]:
df.head(2)

,Unnamed: 0,ClaudeHiakuWeb-raw_rawoutput,ClaudeHiakuWeb-raw_confidence-sen,ClaudeHiakuWeb-raw_confidence-num,ClaudeSonnetWeb-raw_rawoutput,ClaudeSonnetWeb-raw_confidence-sen,ClaudeSonnetWeb-raw_confidence-num,ClaudeOpuWeb-raw_rawoutput,ClaudeOpuWeb-raw_confidence-sen,ClaudeOpuWeb-raw_confidence-num,...,Llama3.1-405B_confidence-num,Llama3.1-70B_rawoutput,Llama3.1-70B_confidence-sen,Llama3.1-70B_confidence-num,Llama3.1-8B_rawoutput,Llama3.1-8B_confidence-sen,Llama3.1-8B_confidence-num,gemma2-27b-it_rawoutput,gemma2-27b-it_confidence-sen,gemma2-27b-it_confidence-num
0,0,"As a seasoned gastroenterologist, I would appr...",My confidence in this decision is 8 out of 10.,8,To approach this complex question from the per...,Confidence rating: 8/10 While colonic stenting...,8,To approach this question as an experienced ga...,My confidence level in this decision is 8 out ...,8,...,9,"As a seasoned gastroenterologist, I will evalu...",(Confidence level: 8/10) (Confidence level: ...,8,"As a seasoned gastroenterologist, I will caref...",**Confidence level: 9/10** I am highly confi...,9,Let's break down this complex case.\n\n**Under...,**Expert Judgment and Confidence:** **Confid...,8
1,1,"As an experienced gastroenterologist, I will t...",My confidence in this decision is 9 out of 10.,9,"As a seasoned gastroenterologist, I will evalu...",I will then choose the most accurate option ba...,not_mentioned,"As an experienced gastroenterologist, I will e...",Confidence rating: 8/10.,8,...,9,"As a seasoned gastroenterologist, I will evalu...","I am highly confident in this decision, rating...",9,"As a seasoned gastroenterologist, I will evalu...",**Confidence rating: 10/10** I am highly con...,10,This is a complex case requiring careful consi...,**Expert Judgment and Confidence:** ** **Co...,9
